# Predicting sentiment from product reviews


The goal of this first notebook is to explore logistic regression and feature engineering with existing GraphLab functions.

In this notebook you will use product review data from Amazon.com to predict whether the sentiments about a product (from its reviews) are positive or negative.

* Use SFrames to do some feature engineering
* Train a logistic regression model to predict the sentiment of product reviews.
* Inspect the weights (coefficients) of a trained logistic regression model.
* Make a prediction (both class and probability) of sentiment for a new product review.
* Given the logistic regression weights, predictors and ground truth labels, write a function to compute the **accuracy** of the model.
* Inspect the coefficients of the logistic regression model and interpret their meanings.
* Compare multiple logistic regression models.

Let's get started!
    
## Fire up GraphLab Create

Make sure you have the latest version of GraphLab Create.

In [20]:
from __future__ import division
import math
import string
import pandas as pd
import numpy as np

# Data preparation

We will use a dataset consisting of baby product reviews on Amazon.com.

In [5]:
products = pd.read_csv('amazon_baby.csv')

Now, let us see a preview of what the dataset looks like.

In [6]:
products

,name,review,rating
0,Planetwise Flannel Wipes,"These flannel wipes are OK, but in my opinion ...",3
1,Planetwise Wipe Pouch,it came early and was not disappointed. i love...,5
2,Annas Dream Full Quilt with 2 Shams,Very soft and comfortable and warmer than it l...,5
3,Stop Pacifier Sucking without tears with Thumb...,This is a product well worth the purchase. I ...,5
4,Stop Pacifier Sucking without tears with Thumb...,All of my kids have cried non-stop when I trie...,5
5,Stop Pacifier Sucking without tears with Thumb...,"When the Binky Fairy came to our house, we did...",5
6,A Tale of Baby's Days with Peter Rabbit,"Lovely book, it's bound tightly so you may not...",4
7,"Baby Tracker&reg; - Daily Childcare Journal, S...",Perfect for new parents. We were able to keep ...,5
8,"Baby Tracker&reg; - Daily Childcare Journal, S...",A friend of mine pinned this product on Pinter...,5
9,"Baby Tracker&reg; - Daily Childcare Journal, S...",This has been an easy way for my nanny to reco...,4


## Build the word count vector for each review

Let us explore a specific example of a baby product.


In [8]:
products.iloc[269]

name      The First Years Massaging Action Teether
review                    A favorite in our house!
rating                                           5
Name: 269, dtype: object

Now, we will perform 2 simple data transformations:

1. Remove punctuation using [Python's built-in](https://docs.python.org/2/library/string.html) string functionality.
2. Transform the reviews into word-counts.

**Aside**. In this notebook, we remove all punctuations for the sake of simplicity. A smarter approach to punctuations would preserve phrases such as "I'd", "would've", "hadn't" and so forth. See [this page](https://www.cis.upenn.edu/~treebank/tokenization.html) for an example of smart handling of punctuations.

def remove_punctuation(text):
    import string
    return text.translate(None, string.punctuation) 

review_without_punctuation = products['review'].apply(remove_punctuation)
products['word_count'] = graphlab.text_analytics.count_words(review_without_punctuation)

In [9]:
#Perform text cleaning by removing punctuations
products['review_clean'] = products['review'].str.replace('[^\w\s]','') 

In [12]:
#fill in N/A's in the review column
products = products.fillna({'review':''})  

Now, let us explore what the sample example above looks like after these 2 transformations. Here, each entry in the **word_count** column is a dictionary where the key is the word and the value is a count of the number of times the word occurs.

products[269]['word_count']

## Extract sentiments

We will **ignore** all reviews with *rating = 3*, since they tend to have a neutral sentiment.

In [13]:
products = products[products['rating'] != 3]
len(products)

166752

Now, we will assign reviews with a rating of 4 or higher to be *positive* reviews, while the ones with rating of 2 or lower are *negative*. For the sentiment column, we use +1 for the positive class label and -1 for the negative class label.

In [14]:
products['sentiment'] = products['rating'].apply(lambda rating : +1 if rating > 3 else -1)
products

,name,review,rating,review_clean,sentiment
1,Planetwise Wipe Pouch,it came early and was not disappointed. i love...,5,it came early and was not disappointed i love ...,1
2,Annas Dream Full Quilt with 2 Shams,Very soft and comfortable and warmer than it l...,5,Very soft and comfortable and warmer than it l...,1
3,Stop Pacifier Sucking without tears with Thumb...,This is a product well worth the purchase. I ...,5,This is a product well worth the purchase I h...,1
4,Stop Pacifier Sucking without tears with Thumb...,All of my kids have cried non-stop when I trie...,5,All of my kids have cried nonstop when I tried...,1
5,Stop Pacifier Sucking without tears with Thumb...,"When the Binky Fairy came to our house, we did...",5,When the Binky Fairy came to our house we didn...,1
6,A Tale of Baby's Days with Peter Rabbit,"Lovely book, it's bound tightly so you may not...",4,Lovely book its bound tightly so you may not b...,1
7,"Baby Tracker&reg; - Daily Childcare Journal, S...",Perfect for new parents. We were able to keep ...,5,Perfect for new parents We were able to keep t...,1
8,"Baby Tracker&reg; - Daily Childcare Journal, S...",A friend of mine pinned this product on Pinter...,5,A friend of mine pinned this product on Pinter...,1
9,"Baby Tracker&reg; - Daily Childcare Journal, S...",This has been an easy way for my nanny to reco...,4,This has been an easy way for my nanny to reco...,1
10,"Baby Tracker&reg; - Daily Childcare Journal, S...",I love this journal and our nanny uses it ever...,4,I love this journal and our nanny uses it ever...,1


Now, we can see that the dataset contains an extra column called **sentiment** which is either positive (+1) or negative (-1).

## Split data into training and test sets

Let's perform a train/test split with 80% of the data in the training set and 20% of the data in the test set. We use `seed=1` so that everyone gets the same result.

train_data, test_data = products.random_split(.8, seed=1)
print len(train_data)
print len(test_data)

In [15]:
#Split into training and test sets    
train_idx = pd.read_json('module-2-assignment-train-idx.json')
test_idx = pd.read_json('module-2-assignment-test-idx.json')
train_data = products.iloc[train_idx.iloc[:,0].values]
test_data = products.iloc[test_idx.iloc[:,0].values]

# Train a sentiment classifier with logistic regression

We will now use logistic regression to create a sentiment classifier on the training data. This model will use the column **word_count** as a feature and the column **sentiment** as the target. We will use `validation_set=None` to obtain same results as everyone else.

**Note:** This line may take 1-2 minutes.

sentiment_model = graphlab.logistic_classifier.create(train_data,
                                                      target = 'sentiment',
                                                      features=['word_count'],
                                                      validation_set=None)

In [16]:
############           Preprocessing            ##############
#Build the word count vector for each review
#bag-of-word features
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer(token_pattern=r'\b\w+\b')

#Use this token pattern to keep single-letter words
#First, learn vocabulary from the training data and assign columns to words
#Then convert the training data into a sparse matrix
#train_data['review_clean'].isnull().sum() > 0 --> there're nulls
train_data = train_data.fillna({'review_clean':''}) #X
train_matrix = vectorizer.fit_transform(train_data['review_clean']) #h(X)

#Second, convert the test data into a sparse matrix, 
#using the same word-column mapping
test_data = test_data.fillna({'review_clean':''})
test_matrix = vectorizer.transform(test_data['review_clean'])

In [17]:
#Train a sentiment classifier with logistic regression
#use the sparse word count matrix (train_matrix) as features and the column 
#sentiment of train_data as the target
from sklearn.linear_model import LogisticRegression
sentiment_model = LogisticRegression()
sentiment_model.fit(train_matrix, train_data['sentiment'])

C:\Users\vwzheng.ARETEINC\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\vwzheng.ARETEINC\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\svm\base.py:922: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False)

**Aside**. You may get a warning to the effect of "Terminated due to numerical difficulties --- this model may not be ideal". It means that the quality metric (to be covered in Module 3) failed to improve in the last iteration of the run. The difficulty arises as the sentiment model puts too much weight on extremely rare words. A way to rectify this is to apply regularization, to be covered in Module 4. Regularization lessens the effect of extremely rare words. For the purpose of this assignment, however, please proceed with the model above.

Now that we have fitted the model, we can extract the weights (coefficients) as an SFrame as follows:

weights = sentiment_model.coefficients
weights.column_names()

In [72]:
del weights
weights = pd.DataFrame({'word': np.insert(vectorizer.get_feature_names(), 
                                          0, 'intercept'),
                        'weight': np.insert(sentiment_model.coef_.flatten(), 
                                            0, sentiment_model.intercept_)})
weights

,word,weight
0,intercept,1.378879e+00
1,0,-1.237001e+00
2,00,1.923607e-04
3,000,2.598827e-02
4,0001,5.983180e-03
5,001,4.279150e-05
6,001cm,8.741242e-07
7,002,2.460233e-03
8,01,2.666444e-01
9,010,2.541052e-01


In [31]:
#positive weights w_j correspond to weights that cause positive sentiment, 
#while negative weights correspond to negative sentiment
#Calculate the number of positive (>= 0, nonnegative) coeffs 
cntnonneg = np.sum(sentiment_model.coef_>=0) + \
            np.sum(sentiment_model.intercept_>0)
print('number of non-negative coefficients including constant: ', cntnonneg)
print(sentiment_model.coef_.shape) #(1, 121741)
num_weights = sentiment_model.coef_.shape[1] + 1
print('number of coefficients including constant: ', num_weights)

number of non-negative coefficients including constant:  85811
(1, 121741)
number of coefficients including constant:  121742


There are a total of `121713` coefficients in the model. Recall from the lecture that positive weights $w_j$ correspond to weights that cause positive sentiment, while negative weights correspond to negative sentiment. 

Fill in the following block of code to calculate how many *weights* are positive ( >= 0). (**Hint**: The `'value'` column in SFrame *weights* must be positive ( >= 0)).

In [33]:
num_positive_weights = np.sum(sentiment_model.coef_ >= 0) + np.sum(sentiment_model.intercept_ >= 0)
num_negative_weights = np.sum(sentiment_model.coef_ < 0) + np.sum(sentiment_model.intercept_ < 0)

print ("Number of positive weights: %s " % num_positive_weights)
print ("Number of negative weights: %s " % num_negative_weights)

Number of positive weights: 85811 
Number of negative weights: 35931 


**Quiz Question:** How many weights are >= 0?

## Making predictions with logistic regression

Now that a model is trained, we can make predictions on the **test data**. In this section, we will explore this in the context of 3 examples in the test dataset.  We refer to this set of 3 examples as the **sample_test_data**.

In [34]:
sample_test_data = test_data[10:13] #index of row: 10, 11, 12
#return 11th row to 13th row
print (sample_test_data['rating'])
sample_test_data

59    5
71    2
91    1
Name: rating, dtype: int64


,name,review,rating,review_clean,sentiment
59,Our Baby Girl Memory Book,Absolutely love it and all of the Scripture in...,5,Absolutely love it and all of the Scripture in...,1
71,Wall Decor Removable Decal Sticker - Colorful ...,Would not purchase again or recommend. The dec...,2,Would not purchase again or recommend The deca...,-1
91,New Style Trailing Cherry Blossom Tree Decal R...,Was so excited to get this product for my baby...,1,Was so excited to get this product for my baby...,-1


Let's dig deeper into the first row of the **sample_test_data**. Here's the full review:

In [36]:
sample_test_data.iloc[0]['review'] #get one specific row using iloc

'Absolutely love it and all of the Scripture in it.  I purchased the Baby Boy version for my grandson when he was born and my daughter-in-law was thrilled to receive the same book again.'

That review seems pretty positive.

Now, let's see what the next row of the **sample_test_data** looks like. As we could guess from the sentiment (-1), the review is quite negative.

In [37]:
sample_test_data.iloc[1]['review']

'Would not purchase again or recommend. The decals were thick almost plastic like and were coming off the wall as I was applying them! The would NOT stick! Literally stayed stuck for about 5 minutes then started peeling off.'

We will now make a **class** prediction for the **sample_test_data**. The `sentiment_model` should predict **+1** if the sentiment is positive and **-1** if the sentiment is negative. Recall from the lecture that the **score** (sometimes called **margin**) for the logistic regression model  is defined as:

$$
\mbox{score}_i = \mathbf{w}^T h(\mathbf{x}_i)
$$ 

where $h(\mathbf{x}_i)$ represents the features for example $i$.  We will write some code to obtain the **scores** using GraphLab Create. For each row, the **score** (or margin) is a number in the range **[-inf, inf]**.

scores = sentiment_model.predict(sample_test_data, output_type='margin')
print scores

In [38]:
#The sentiment_model should predict +1 if the sentiment is positive
#-1 if the sentiment is negative
sample_test_matrix = vectorizer.transform(sample_test_data['review_clean'])
#calculate the score of each data point with decision_function()
scores = sentiment_model.decision_function(sample_test_matrix) #WTransposeh(X)
print (scores)

[  5.60242759  -3.15471267 -10.4208805 ]


### Predicting sentiment

These scores can be used to make class predictions as follows:

$$
\hat{y} = 
\left\{
\begin{array}{ll}
      +1 & \mathbf{w}^T h(\mathbf{x}_i) > 0 \\
      -1 & \mathbf{w}^T h(\mathbf{x}_i) \leq 0 \\
\end{array} 
\right.
$$

Using scores, write code to calculate $\hat{y}$, the class predictions:

In [40]:
#Prediciting Sentiment
#make class predictions from scores
def predictions(scores):
    """ make class predictions
    """
    preds = []
    for score in scores:
        if score > 0:
            pred = 1
        else:
            pred = -1
        preds.append(pred)
    return preds
sample_test_predictions = predictions(scores)
print('sample prediction of class: ', sample_test_predictions)

sample prediction of class:  [1, -1, -1]


Run the following code to verify that the class predictions obtained by your calculations are the same as that obtained from GraphLab Create.

print "Class predictions according to GraphLab Create:" 
print sentiment_model.predict(sample_test_data)

In [42]:
#Use predict() to check
sentiment_model.predict(sample_test_matrix)
print ("Class predictions according to Logistic Regression:" )
print (sentiment_model.predict(sample_test_matrix))

Class predictions according to Logistic Regression:
[ 1 -1 -1]


**Checkpoint**: Make sure your class predictions match with the one obtained from GraphLab Create.

### Probability predictions

Recall from the lectures that we can also calculate the probability predictions from the scores using:
$$
P(y_i = +1 | \mathbf{x}_i,\mathbf{w}) = \frac{1}{1 + \exp(-\mathbf{w}^T h(\mathbf{x}_i))}.
$$

Using the variable **scores** calculated previously, write code to calculate the probability that a sentiment is positive using the above formula. For each row, the probabilities should be a number in the range **[0, 1]**.

In [43]:
#Probability Predictions
#calculate the probability predictions from scores as the probability 
#that a sentiment is positive
def probability(scores):
    probs = []
    for score in scores:
        probs.append(1/(1+np.exp(-score)))
    return probs
sample_test_probabilities = probability(scores)
print('sample probability: ', sample_test_probabilities)

sample probability:  [0.9963246602299167, 0.040905987014292565, 2.980273751526485e-05]


**Checkpoint**: Make sure your probability predictions match the ones obtained from GraphLab Create.

print "Class predictions according to GraphLab Create:" 
print sentiment_model.predict(sample_test_data, output_type='probability')

In [45]:
print ("Class (positive) prediction probabilities according to Logistic Regression:" )
print(sentiment_model.predict_proba(sample_test_matrix)[:,1])   

Class prediction probabilities according to Logistic Regression:
[9.96324660e-01 4.09059870e-02 2.98027375e-05]


** Quiz Question:** Of the three data points in **sample_test_data**, which one (first, second, or third) has the **lowest probability** of being classified as a positive review?

#### 13th record of test data set or the third record of sample test data

# Find the most positive (and negative) review

We now turn to examining the full test dataset, **test_data**, and use GraphLab Create to form predictions on all of the test data points for faster performance.

Using the `sentiment_model`, find the 20 reviews in the entire **test_data** with the **highest probability** of being classified as a **positive review**. We refer to these as the "most positive reviews."

To calculate these top-20 reviews, use the following steps:
1.  Make probability predictions on **test_data** using the `sentiment_model`. (**Hint:** When you call `.predict` to make predictions on the test data, use option `output_type='probability'` to output the probability rather than just the most likely class.)
2.  Sort the data according to those predictions and pick the top 20. (**Hint:** You can use the `.topk` method on an SFrame to find the top k rows sorted according to the value of a specified column.)

In [47]:
#Find the most positive (and negative) review
#find the 20 reviews in the test_data with the highest probability of 
#being classified as a positive review 
test_probs = sentiment_model.predict_proba(test_matrix)
#return indices of sorted values in ascending order
positive20 = np.argsort(-test_probs[:,1])[:20]
#find products based on indices
positive20_test = test_data.iloc[positive20]
positive20_test

,name,review,rating,review_clean,sentiment
100166,"Infantino Wrap and Tie Baby Carrier, Black Blu...",I bought this carrier when my daughter was abo...,5,I bought this carrier when my daughter was abo...,1
50315,"P'Kolino Silly Soft Seating in Tias, Green",I've purchased both the P'Kolino Little Reader...,4,Ive purchased both the PKolino Little Reader C...,1
168697,Graco FastAction Fold Jogger Click Connect Str...,Graco's FastAction Jogging Stroller definitely...,5,Gracos FastAction Jogging Stroller definitely ...,1
119182,Roan Rocco Classic Pram Stroller 2-in-1 with B...,Great Pram Rocco!!!!!!I bought this pram from ...,5,Great Pram RoccoI bought this pram from Europe...,1
133651,"Britax 2012 B-Agile Stroller, Red",[I got this stroller for my daughter prior to ...,4,I got this stroller for my daughter prior to t...,1
80155,"Simple Wishes Hands-Free Breastpump Bra, Pink,...","I just tried this hands free breastpump bra, a...",5,I just tried this hands free breastpump bra an...,1
87017,Baby Einstein Around The World Discovery Center,I am so HAPPY I brought this item for my 7 mon...,5,I am so HAPPY I brought this item for my 7 mon...,1
147949,"Baby Jogger City Mini GT Single Stroller, Shad...","Amazing, Love, Love, Love it !!! All 5 STARS a...",5,Amazing Love Love Love it All 5 STARS all the...,1
140816,"Diono RadianRXT Convertible Car Seat, Plum",I bought this seat for my tall (38in) and thin...,5,I bought this seat for my tall 38in and thin 2...,1
168081,Buttons Cloth Diaper Cover - One Size - 8 Colo...,"We are big Best Bottoms fans here, but I wante...",4,We are big Best Bottoms fans here but I wanted...,1


**Quiz Question**: Which of the following products are represented in the 20 most positive reviews? [multiple choice]


Now, let us repeat this exercise to find the "most negative reviews." Use the prediction probabilities to find the  20 reviews in the **test_data** with the **lowest probability** of being classified as a **positive review**. Repeat the same steps above but make sure you **sort in the opposite order**.

In [48]:
#find the 20 reviews in the test_data with the lowest probability of 
#being classified as a positive review
negative20 = np.argsort(test_probs[:,1])[:20]
negative20_test = test_data.iloc[negative20]
negative20_test

,name,review,rating,review_clean,sentiment
16042,Fisher-Price Ocean Wonders Aquarium Bouncer,We have not had ANY luck with Fisher-Price pro...,2,We have not had ANY luck with FisherPrice prod...,-1
120209,Levana Safe N'See Digital Video Baby Monitor w...,This is the first review I have ever written o...,1,This is the first review I have ever written o...,-1
77072,Safety 1st Exchangeable Tip 3 in 1 Thermometer,I thought it sounded great to have different t...,1,I thought it sounded great to have different t...,-1
48694,Adiri BPA Free Natural Nurser Ultimate Bottle ...,I will try to write an objective review of the...,2,I will try to write an objective review of the...,-1
155287,VTech Communications Safe &amp; Sounds Full Co...,"This is my second video monitoring system, the...",1,This is my second video monitoring system the ...,-1
94560,The First Years True Choice P400 Premium Digit...,Note: we never installed batteries in these un...,1,Note we never installed batteries in these uni...,-1
53207,Safety 1st High-Def Digital Monitor,We bought this baby monitor to replace a diffe...,1,We bought this baby monitor to replace a diffe...,-1
81332,Cloth Diaper Sprayer--styles may vary,I bought this sprayer out of desperation durin...,1,I bought this sprayer out of desperation durin...,-1
113995,Motorola Digital Video Baby Monitor with Room ...,DO NOT BUY THIS BABY MONITOR!I purchased this ...,1,DO NOT BUY THIS BABY MONITORI purchased this m...,-1
10677,Philips AVENT Newborn Starter Set,"It's 3am in the morning and needless to say, t...",1,Its 3am in the morning and needless to say thi...,-1


**Quiz Question**: Which of the following products are represented in the 20 most negative reviews?  [multiple choice]

## Compute accuracy of the classifier

We will now evaluate the accuracy of the trained classifier. Recall that the accuracy is given by


$$
\mbox{accuracy} = \frac{\mbox{# correctly classified examples}}{\mbox{# total examples}}
$$

This can be computed as follows:

* **Step 1:** Use the trained model to compute class predictions (**Hint:** Use the `predict` method)
* **Step 2:** Count the number of data points when the predicted class labels match the ground truth labels (called `true_labels` below).
* **Step 3:** Divide the total number of correct predictions by the total number of data points in the dataset.

Complete the function below to compute the classification accuracy:

In [49]:
def get_classification_accuracy(model, data, true_labels):
    # First get the predictions
    ## YOUR CODE HERE
    predictions = model.predict(data)
    
    # Compute the number of correctly classified examples
    ## YOUR CODE HERE
    num_correct = np.sum(predictions == true_labels)

    # Then compute accuracy by dividing num_correct by total number of examples
    ## YOUR CODE HERE
    accuracy = num_correct / len(predictions)
    
    return accuracy

Now, let's compute the classification accuracy of the **sentiment_model** on the **test_data**.

In [51]:
print(get_classification_accuracy(sentiment_model, test_matrix, test_data['sentiment']))

0.9323254139668826


**Quiz Question**: What is the accuracy of the **sentiment_model** on the **test_data**? Round your answer to 2 decimal places (e.g. 0.76).

**Quiz Question**: Does a higher accuracy value on the **training_data** always imply that the classifier is better?

#### No, a higher accuracy value on the training data does not always imply that the classifier is better.

In [52]:
print(get_classification_accuracy(sentiment_model, train_matrix, train_data['sentiment']))

0.9677699826107813


## Learn another classifier with fewer words

There were a lot of words in the model we trained above. We will now train a simpler logistic regression model using only a subset of words that occur in the reviews. For this assignment, we selected a 20 words to work with. These are:

In [54]:
significant_words = ['love', 'great', 'easy', 'old', 'little', 'perfect', 'loves', 
      'well', 'able', 'car', 'broke', 'less', 'even', 'waste', 'disappointed', 
      'work', 'product', 'money', 'would', 'return']

In [55]:
len(significant_words)

20

For each review, we will use the **word_count** column and trim out all words that are **not** in the **significant_words** list above. We will use the [SArray dictionary trim by keys functionality]( https://dato.com/products/create/docs/generated/graphlab.SArray.dict_trim_by_keys.html). Note that we are performing this on both the training and test set.

train_data['word_count_subset'] = train_data['word_count'].dict_trim_by_keys(significant_words, exclude=False)
test_data['word_count_subset'] = test_data['word_count'].dict_trim_by_keys(significant_words, exclude=False)

In [56]:
#redo vectorizer with limit to 20 words
vectorizer_subset = CountVectorizer(vocabulary=significant_words) 
train_matrix_subset = vectorizer_subset.fit_transform(train_data['review_clean'])
test_matrix_subset = vectorizer_subset.transform(test_data['review_clean'])

Let's see what the first example of the dataset looks like:

In [58]:
train_data.iloc[0]['review']

'it came early and was not disappointed. i love planet wise bags and now my wipe holder. it keps my osocozy wipes moist and does not leak. highly recommend it.'

The **word_count** column had been working with before looks like the following:

print train_data[0]['word_count']

Since we are only working with a subset of these words, the column **word_count_subset** is a subset of the above dictionary. In this example, only 2 `significant words` are present in this review.

print train_data[0]['word_count_subset']

## Train a logistic regression model on a subset of data

We will now build a classifier with **word_count_subset** as the feature and **sentiment** as the target. 

simple_model = graphlab.logistic_classifier.create(train_data,
                                                   target = 'sentiment',
                                                   features=['word_count_subset'],
                                                   validation_set=None)
simple_model

In [59]:
#Train a logistic regression model on a subset of data
simple_model = LogisticRegression()
simple_model.fit(train_matrix_subset, train_data['sentiment'])

C:\Users\vwzheng.ARETEINC\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False)

We can compute the classification accuracy using the `get_classification_accuracy` function you implemented earlier.

In [60]:
get_classification_accuracy(simple_model, test_matrix_subset, test_data['sentiment'])

0.8693604511639069

Now, we will inspect the weights (coefficients) of the **simple_model**:

simple_model.coefficients

In [75]:
simple_weights = pd.DataFrame({'word': np.insert(significant_words, 0, 'intercept'),
                               'weight': np.insert(simple_model.coef_.flatten(), 0, simple_model.intercept_)})
simple_weights

,word,weight
0,intercept,1.299374
1,love,1.363690
2,great,0.944000
3,easy,1.192538
4,old,0.085513
5,little,0.520186
6,perfect,1.509812
7,loves,1.673074
8,well,0.503760
9,able,0.190909


Let's sort the coefficients (in descending order) by the **value** to obtain the coefficients with the most positive effect on the sentiment.

simple_model.coefficients.sort('value', ascending=False).print_rows(num_rows=21)

In [64]:
simple_coefs.sort_values('coefficient', ascending = False)

,word,coefficient
7,loves,1.673074
6,perfect,1.509812
1,love,1.363690
0,constant,1.299374
3,easy,1.192538
2,great,0.944000
5,little,0.520186
8,well,0.503760
9,able,0.190909
4,old,0.085513


**Quiz Question**: Consider the coefficients of **simple_model**. There should be 21 of them, an intercept term + one for each word in **significant_words**. How many of the 20 coefficients (corresponding to the 20 **significant_words** and *excluding the intercept term*) are positive for the `simple_model`?

In [65]:
print(np.sum(simple_model.coef_ >= 0))

10


**Quiz Question**: Are the positive words in the **simple_model** (let us call them `positive_significant_words`) also positive words in the **sentiment_model**?

In [76]:
del simple_coefs
simple_coefs = pd.DataFrame({'word':significant_words,
                             'coefficient':simple_model.coef_.flatten()})
sentiment_coefs = pd.DataFrame({'word':vectorizer.get_feature_names(),
                                'coefficient':sentiment_model.coef_.flatten()})
positive_significant_words = simple_coefs[simple_coefs['coefficient'] >= 0]['word']
#subset of words not in significant_words_pos
#sentiment_coefs[~sentiment_coefs.word.isin(significant_words_pos)]    
positive_significant_words_sentiment = sentiment_coefs[sentiment_coefs.word.isin(positive_significant_words)]
print(len(positive_significant_words), len(positive_significant_words_sentiment))
print(positive_significant_words_sentiment)

10 10
           word  coefficient
7399       able     0.389626
22139       car     0.124945
37657      easy     1.358114
48806     great     1.230278
62621    little     0.637371
63586      love     1.575775
63665     loves     1.518048
74129       old     0.051315
79004   perfect     1.860013
117934     well     0.539984


# Comparing models

We will now compare the accuracy of the **sentiment_model** and the **simple_model** using the `get_classification_accuracy` method you implemented above.

First, compute the classification accuracy of the **sentiment_model** on the **train_data**:

In [78]:
get_classification_accuracy(sentiment_model, train_matrix, train_data['sentiment'])

0.9677699826107813

Now, compute the classification accuracy of the **simple_model** on the **train_data**:

In [80]:
get_classification_accuracy(simple_model, train_matrix_subset, train_data['sentiment'])

0.8668225700065959

**Quiz Question**: Which model (**sentiment_model** or **simple_model**) has higher accuracy on the TRAINING set?

Now, we will repeat this exercise on the **test_data**. Start by computing the classification accuracy of the **sentiment_model** on the **test_data**:

In [81]:
get_classification_accuracy(sentiment_model, test_matrix, test_data['sentiment'])

0.9323254139668826

Next, we will compute the classification accuracy of the **simple_model** on the **test_data**:

In [82]:
get_classification_accuracy(simple_model, test_matrix_subset, test_data['sentiment'])

0.8693604511639069

**Quiz Question**: Which model (**sentiment_model** or **simple_model**) has higher accuracy on the TEST set?

## Baseline: Majority class prediction

It is quite common to use the **majority class classifier** as the a baseline (or reference) model for comparison with your classifier model. The majority classifier model predicts the majority class for all data points. At the very least, you should healthily beat the majority class classifier, otherwise, the model is (usually) pointless.

What is the majority class in the **train_data**?

In [83]:
num_positive  = (train_data['sentiment'] == +1).sum()
num_negative = (train_data['sentiment'] == -1).sum()
print (num_positive)
print (num_negative)

112164
21252


Now compute the accuracy of the majority class classifier on **test_data**.

**Quiz Question**: Enter the accuracy of the majority class classifier model on the **test_data**. Round your answer to two decimal places (e.g. 0.76).

In [86]:
#Majority class classifier model: assuming making all predictions +1
#since +1's (positive reviews) are the majority class
#accuracy = real positive reviews in test / predicted positive reviews in test
#accuracy = precision = TP/PredP, PredP = len(test), TP = num_positive_test
num_positive_test = len(test_data[test_data['sentiment']>0])
num_negative_test = len(test_data[test_data['sentiment']<0])#len(test_data)-num_positive_test
majority_accuracy = num_positive_test/(len(test_data))
print(num_positive_test, majority_accuracy)

28095 0.8427825773938085


**Quiz Question**: Is the **sentiment_model** definitely better than the majority class classifier (the baseline)?